In [1]:
# !pip install d3rlpy

In [2]:
import matplotlib.pyplot as plt
!matplotlib inline

/bin/bash: matplotlib: command not found


In [3]:
import pandas as pd
import d3rlpy
import numpy as np
import pandas as pd

train = pd.read_csv('train_data.csv')
test = pd.read_csv('test_data.csv')

In [4]:
def previous_rate_process(data, mean=None):
    data['Previous_Rate_Known'] = data['Previous_Rate'] != -10
    prev_rate = data['Previous_Rate'].copy()
    if mean:
        prev_rate_mean = mean
    else:
        prev_rate_mean = prev_rate[prev_rate != -10].mean()
    prev_rate[prev_rate == -10] = prev_rate_mean
    data['Previous_Rate'] = prev_rate
    return prev_rate_mean

prev_rate_mean = previous_rate_process(train)
previous_rate_process(test, prev_rate_mean)
train.head()

,Tier,FICO,Term,Amount,Previous_Rate,Competition_rate,Rate,Cost_Funds,Partner Bin,Car_Type_N,Car_Type_R,Car_Type_U,Accept,Previous_Rate_Known
0,2,725,72,30500.0,5.000000,6.09,4.99,1.1200,2,0,1,0,0,True
1,1,739,60,25995.0,7.625269,4.79,4.79,1.9590,2,0,0,1,0,False
2,1,781,60,39000.0,7.625269,4.25,4.25,1.1200,2,1,0,0,0,False
3,2,718,72,30000.0,7.625269,5.39,5.44,1.3363,1,1,0,0,1,False
4,2,703,72,38000.0,7.625269,5.75,6.29,1.1338,1,0,0,1,0,False


In [5]:
train.describe()

,Tier,FICO,Term,Amount,Previous_Rate,Competition_rate,Rate,Cost_Funds,Partner Bin,Car_Type_N,Car_Type_R,Car_Type_U,Accept
count,145659.000000,145659.000000,145659.000000,145659.000000,145659.000000,145659.000000,145659.000000,145659.000000,145659.000000,145659.000000,145659.000000,145659.000000,145659.000000
mean,1.928429,726.695556,56.822441,25996.819760,7.625269,4.807957,5.623998,1.329209,2.029473,0.571815,0.226865,0.201320,0.220041
std,1.050519,44.727756,11.201037,11125.968255,1.323102,0.586018,1.545418,0.278368,0.911097,0.494817,0.418806,0.400988,0.414276
min,1.000000,594.000000,36.000000,4526.620000,3.000000,2.990000,2.450000,1.020000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,692.000000,48.000000,17753.450000,7.625269,4.390000,4.490000,1.110000,1.000000,0.000000,0.000000,0.000000,0.000000
50%,2.000000,726.000000,60.000000,25000.000000,7.625269,4.790000,5.090000,1.262500,2.000000,1.000000,0.000000,0.000000,0.000000
75%,3.000000,762.000000,60.000000,33000.000000,7.625269,5.190000,6.390000,1.419400,3.000000,1.000000,0.000000,0.000000,0.000000
max,4.000000,852.000000,72.000000,100000.000000,24.000000,6.450000,15.530000,2.127000,3.000000,1.000000,1.000000,1.000000,1.000000


In [6]:
from sklearn.linear_model import LogisticRegression

X = train[['Tier', 'FICO', 'Term', 'Amount', 'Previous_Rate', 'Competition_rate',
           'Cost_Funds', 'Partner Bin', 'Car_Type_N', 'Car_Type_R', 'Car_Type_U', 'Rate']].values
y = train['Accept'].values

clf = LogisticRegression(random_state=0, penalty='none', fit_intercept=True, multi_class='ovr', n_jobs = 4).fit(X, y)
probs_train = clf.predict_proba(X)[:,1]

/home/user/conda/envs/ldm/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1183: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(


In [7]:
from sklearn.model_selection import train_test_split

train_data, eval_data = train_test_split(train[:10000], test_size=0.15, shuffle=True)

In [8]:
observations = train_data[['Tier', 'FICO', 'Term', 'Amount', 'Previous_Rate', 'Competition_rate',
                           'Cost_Funds', 'Partner Bin', 'Car_Type_N', 'Car_Type_R', 'Car_Type_U']]
actions = train_data['Rate']

terminals = np.zeros_like(actions)
terminals[-1] = 1

In [9]:
observations = observations.values
actions = actions.values
actions = actions.reshape(-1,1)

In [10]:
def prob_accept(action, state):
    if isinstance(action, (int, float)):
        action = np.array(action)
    action = action.reshape(-1,1)
    if len(state.shape)==1:
        state = state.reshape(1,-1)
    x = np.concatenate((state, action),axis=1)
    probs_pred = clf.predict_proba(x)[:,1]
    return probs_pred

prob_accept(actions, observations)

array([0.62314798, 0.48864422, 0.48851737, ..., 0.21507262, 0.72913991,
       0.02747331])

In [11]:
def p_default(state):
    fico = state[:, 1]
    default_prob = fico.copy()
    default_prob[default_prob < 500] = 0.41
    default_prob[default_prob >= 750] = 0.01
    default_prob[default_prob >= 700] = 0.044
    default_prob[default_prob >= 650] = 0.089
    default_prob[default_prob >= 600] = 0.158
    default_prob[default_prob >= 550] = 0.225
    default_prob[default_prob >= 500] = 0.284
    return default_prob
    # return np.random.random(size=fico.shape)

def reward(action, state, risk_free = 0.04, loss_ratio=0.5):

    if len(state.shape)==1:
        state = state.reshape(1,-1)
    p_Accept= prob_accept(action, state)

    Sum_loan = state[:,3]
    Term = state[:,2]/12

    p_return = p_default(state)

    loss_given_default = loss_ratio*Sum_loan
    action_rate = action/100
    reward = (p_Accept*(Sum_loan*p_return*((1+action_rate)**Term-(1+risk_free)**Term)-(1-p_return)*loss_given_default))/Sum_loan

    return reward

def reward_vec(action, state, risk_free = 0.2, loss_ratio=0.5):
    if len(state.shape)==1:
        state = state.reshape(1,-1)

    p_Accept= prob_accept(action, state)

    Sum_loan = state[:,3]
    Term = state[:,2]/12

    p_return = p_default(state)

    loss_given_default = loss_ratio*Sum_loan
    action = action/100
    reward = (p_Accept*(Sum_loan*p_return*((1+action)**Term-(1+risk_free)**Term)-(1-p_return)*loss_given_default))/Sum_loan

    return reward
#what is a risk-free rate??
rewards = reward_vec(actions, observations, risk_free = 0.04, loss_ratio=0.5)

print(rewards)
# action = 0.3
# state  = observations[7,:]
# print(state.shape)
# rewardina = reward(action, state, risk_free = 0.2, loss_ratio=0.5)

[[-0.3081207  -0.2324073  -0.2322023  ... -0.09739541 -0.34719793
  -0.01359111]
 [-0.30730391 -0.22958914 -0.22901206 ... -0.09605458 -0.34402302
  -0.01357186]
 [-0.3070353  -0.22866235 -0.22795624 ... -0.09562471 -0.3429921
  -0.01356569]
 ...
 [-0.30762566 -0.23069929 -0.23027251 ... -0.09657652 -0.3452663
  -0.01357935]
 [-0.30812825 -0.23243333 -0.23223162 ... -0.09740805 -0.34722756
  -0.01359129]
 [-0.30846189 -0.23358451 -0.2335254  ... -0.09797179 -0.34854331
  -0.01359938]]


In [12]:
import gym

class Env_rl_bank(gym.Env):
    def __init__(self, dataset):
        super(Env_rl_bank, self).__init__()
        self.dataset = dataset
        self.curr_state = 0
        self.n_observation = dataset.shape[0]
        self.observation_tensor = dataset[['Tier', 'FICO', 'Term', 'Amount', 'Previous_Rate', 'Competition_rate',
        'Cost_Funds', 'Partner Bin', 'Car_Type_N', 'Car_Type_R','Car_Type_U']].values

        self.action_space = gym.spaces.Box(low=np.array([0]), high=np.array([100]), dtype=np.float16)

        self.observation_space = gym.spaces.Box(low = -10*np.ones(11)[None,:], high =  10e+10*np.ones(11)[None,:],
                                               shape=(1,11), dtype=np.float16)
    def step(self, action):
        #update state
        #return rewarn
        #done is an indicator that the whole dataset is
        # next_s, r, done, _ = env.step(a)
        state  =  self.observation_tensor[self.curr_state]
        if self.curr_state >= self.n_observation-2:
            done = True
        else:

            done = False
        self.curr_state +=1
        next_s = self.observation_tensor[self.curr_state]
        rew = reward_vec(action, state, risk_free = 0.04, loss_ratio=0.5)

        info = f'reward = {rew}, at state is done = {done}'

        return next_s, rew, done, info

    def reset(self):
        self.curr_state = 0
        state  =  self.observation_tensor[self.curr_state]
        return state
#     def sample(self, ...):
#         return


In [13]:
d3rlpy.__version__

'1.1.1'

In [14]:
# prepare dataset
#dataset, env = d3rlpy.datasets.get_d4rl('hopper-medium-v0')

# prepare algorithm
#do not forget  about cql parametres in a paper
cql = d3rlpy.algos.CQL(use_gpu=True, n_steps=5,
                       batch_size = 256,
                        hidden_units = [64, 64, 64, 64], weight_decay =0.0001,
gamma = 0.999, n_critics =2,alpha_threshold=10,conservative_weight=5,)
                       

       
                    

env  = Env_rl_bank(train_data)
terminals = np.random.randint(2, size = actions.shape[0])

dataset = d3rlpy.dataset.MDPDataset(
    observations=observations,
    actions=actions,
    rewards=rewards,
    terminals=terminals,

)
eval_obs = eval_data[['Tier', 'FICO', 'Term', 'Amount', 'Previous_Rate', 'Competition_rate',
        'Cost_Funds', 'Partner Bin', 'Car_Type_N', 'Car_Type_R','Car_Type_U']].values
eval_actions = eval_data['Rate'].values.reshape(-1,1)

eval_rewards = reward_vec(eval_actions, eval_obs, risk_free = 0.04, loss_ratio=0.5)
eval_terminals = np.zeros_like(eval_actions)

eval_dataset = d3rlpy.dataset.MDPDataset(
    observations=eval_obs,
    actions=eval_actions,
    rewards=eval_rewards,
    terminals=eval_terminals,)


print(dataset.get_action_size())
#print(dataset.episodes)
# correct format for observations, actions, rewards, terminals
# observation = [...,...,..,]
# rewards = reward()
# actions =interest rate path
#terminals = 

# First of all, you need to prepare your logged data. 
# In this tutorial, let’s use randomly generated data.
# terminals represents the last step of episodes. 
# If terminals[i] == 1.0, i-th step is the terminal state. 
# Otherwise you need to set zeros for non-terminal states.

# train
cql.fit(
    dataset,
    eval_episodes=dataset,
    n_epochs=20,

    scorers={'environment': d3rlpy.metrics.evaluate_on_environment(env),
            'td_error': d3rlpy.metrics.td_error_scorer,},)

2023-10-22 11:53:40 [warning  ] Unused arguments are passed.   hidden_units=[64, 64, 64, 64] weight_decay=0.0001


/home/user/conda/envs/ldm/lib/python3.8/site-packages/gym/spaces/box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float16
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
/home/user/conda/envs/ldm/lib/python3.8/site-packages/gym/spaces/box.py:129: RuntimeWarning: overflow encountered in cast
  self.high = high.astype(self.dtype)


1
2023-10-22 11:53:40 [debug    ] RoundIterator is selected.
2023-10-22 11:53:40 [info     ] Directory is created at d3rlpy_logs/CQL_20231022115340
2023-10-22 11:53:40 [debug    ] Building models...
2023-10-22 11:53:43 [debug    ] Models have been built.
2023-10-22 11:53:43 [info     ] Parameters are saved to d3rlpy_logs/CQL_20231022115340/params.json params={'action_scaler': None, 'actor_encoder_factory': {'type': 'default', 'params': {'activation': 'relu', 'use_batch_norm': False, 'dropout_rate': None}}, 'actor_learning_rate': 0.0001, 'actor_optim_factory': {'optim_cls': 'Adam', 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False}, 'alpha_learning_rate': 0.0001, 'alpha_optim_factory': {'optim_cls': 'Adam', 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False}, 'alpha_threshold': 10, 'batch_size': 256, 'conservative_weight': 5, 'critic_encoder_factory': {'type': 'default', 'params': {'activation': 'relu', 'use_batch_norm': False, 'dropout_rate'

Epoch 1/20:   0%|          | 0/33 [00:00<?, ?it/s]

2023-10-22 11:55:36 [info     ] CQL_20231022115340: epoch=1 step=33 epoch=1 metrics={'time_sample_batch': 0.0004996241945208925, 'time_algorithm_update': 0.05783060102751761, 'temp_loss': 1.6237169468041621, 'temp': 0.9983071594527273, 'alpha_loss': -5.713383327830922, 'alpha': 1.0015706546378857, 'critic_loss': 10.68881232810743, 'actor_loss': -0.2387076055235935, 'time_step': 0.05842078093326453, 'environment': -2340.686983344406, 'td_error': 2.746102146156842} step=33
2023-10-22 11:55:36 [info     ] Model parameters are saved to d3rlpy_logs/CQL_20231022115340/model_33.pt


...

Epoch 19/20:   0%|          | 0/33 [00:00<?, ?it/s]

2023-10-22 12:30:35 [info     ] CQL_20231022115340: epoch=19 step=627 epoch=19 metrics={'time_sample_batch': 0.0004314870545358369, 'time_algorithm_update': 0.028003974394364792, 'temp_loss': -1.7222281513792095, 'temp': 0.9846776001381151, 'alpha_loss': 461.1324990012429, 'alpha': 0.905869747653152, 'critic_loss': -325.61476967551494, 'actor_loss': 139.36203211004084, 'time_step': 0.02850619951883952, 'environment': -1874.6092434976824, 'td_error': 6751.576025436997} step=627
2023-10-22 12:30:35 [info     ] Model parameters are saved to d3rlpy_logs/CQL_20231022115340/model_627.pt


Epoch 20/20:   0%|          | 0/33 [00:00<?, ?it/s]

2023-10-22 12:32:28 [info     ] CQL_20231022115340: epoch=20 step=660 epoch=20 metrics={'time_sample_batch': 0.0004469625877611565, 'time_algorithm_update': 0.029152516162756718, 'temp_loss': -1.9584094972321482, 'temp': 0.9895264452153986, 'alpha_loss': 548.5615983442826, 'alpha': 0.8990937814568029, 'critic_loss': -337.4928329930161, 'actor_loss': 169.1609797622218, 'time_step': 0.029675881067911785, 'environment': -1874.6092434976824, 'td_error': 9328.10997401086} step=660
2023-10-22 12:32:28 [info     ] Model parameters are saved to d3rlpy_logs/CQL_20231022115340/model_660.pt


[(1,
  {'time_sample_batch': 0.0004996241945208925,
   'time_algorithm_update': 0.05783060102751761,
   'temp_loss': 1.6237169468041621,
   'temp': 0.9983071594527273,
   'alpha_loss': -5.713383327830922,
   'alpha': 1.0015706546378857,
   'critic_loss': 10.68881232810743,
   'actor_loss': -0.2387076055235935,
   'time_step': 0.05842078093326453,
   'environment': -2340.686983344406,
   'td_error': 2.746102146156842}),
 (2,
  {'time_sample_batch': 0.0005211974635268703,
   'time_algorithm_update': 0.03426418159947251,
   'temp_loss': 1.6371877590815227,
   'temp': 0.9950189861384305,
   'alpha_loss': -1.9886429960077459,
   'alpha': 1.0039760343956226,
   'critic_loss': 11.551553653948234,
   'actor_loss': 0.3965475062529246,
   'time_step': 0.03487339164271499,
   'environment': -2340.686983344406,
   'td_error': 2.4598990317365392}),
 (3,
  {'time_sample_batch': 0.00044319123932809543,
   'time_algorithm_update': 0.02808962446270567,
   'temp_loss': 1.6321363593592788,
   'temp': 0.9

# Inference

In [18]:
from sklearn.model_selection import train_test_split
test_data = pd.read_csv('test_data.csv')

In [32]:
# use policy  and collect reward??
# How to check???

test_actions = test_data['Rate'].values
test_states = test_data[['Tier', 'FICO', 'Term', 'Amount', 'Previous_Rate', 'Competition_rate',
        'Cost_Funds', 'Partner Bin', 'Car_Type_N', 'Car_Type_R','Car_Type_U']].values
hist_rewards_test = reward_vec(test_actions, test_states, risk_free = 0.04, loss_ratio=0.5)
env  = Env_rl_bank(test_data)
state = env.reset()
rewards_cql = []
for step_i in range(len(test_data)):
    actions = cql.predict(state)
    next_state, rew, done, info = env.step(test_actions)
    rewards_cql.append(rew)
    state = next_state




/home/user/conda/envs/ldm/lib/python3.8/site-packages/gym/spaces/box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float16
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
/home/user/conda/envs/ldm/lib/python3.8/site-packages/gym/spaces/box.py:129: RuntimeWarning: overflow encountered in cast
  self.high = high.astype(self.dtype)


AssertionError: Input must have batch dimension.

In [29]:
cum_sum_rew_cql = np.cumsum(rewards_cql)
cum_sum_rew_hist = np.cumsum(hist_rewards_test)

In [30]:
cum_sum_rew_hist, cum_sum_rew_cql

(array([-1.47296613e-03, -1.14274973e-01, -1.19916873e-01, ...,
        -2.77118592e+03, -2.77119159e+03, -2.77133017e+03]),
 array([], dtype=float64))